# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score, classification_report

# Reading Input

In [ ]:
customer_info_df = pd.read_csv('/content/drive/MyDrive/Upay/UPAY Data - CI_test.csv')
transaction_history_df = pd.read_csv('/content/drive/MyDrive/Upay/UPAY Data - TH_test.csv')

In [ ]:
customer_info_df

,Name,Wallet No,Address,Gender,Balance,Age,Occupation,Loan Approval
0,Ali Bordhi Khan,1376318942,Rural,Male,5481.61,31,Engineer,UC
1,Jagat Seth,1376318871,Urban,Male,8421.55,43,Business,Eligible
2,Ghoseti Begum,1974318972,Urban,Female,212.59,45,NGO,Not Eligible
3,Mir Jafar,1876318879,Urban,Male,6408.06,25,Student,UC
4,Robert Clive,1878318891,Urban,Male,15840.49,54,Business,Eligible


In [ ]:
transaction_history_df

,Wallet No,DateTime,Transaction Type,Amount,Transaction ID,Balance
0,1376318942,4/7/2023 9:26,Pay Bill,1750.76,#0170KMAGWBF,3000.00
1,1376318942,9/6/2023 9:32,Cash In,7696.90,#013ZCDRI9O5,10696.90
2,1376318942,9/24/2023 0:34,Cash Out,4144.81,#017V33OMD6B,6552.09
3,1376318942,12/26/2022 11:40,Pay Bill,1387.33,#014K3P28SKM,5164.76
4,1376318942,8/22/2023 17:20,Received Money,1812.39,#01V2P45XS9Q,6977.15
...,...,...,...,...,...,...
301,1878318891,9/23/2023 10:11,Received Money,4612.36,#01955ET5163,17753.73
302,1878318891,9/25/2023 7:05,Make Payment,3218.51,#01VZQPYVGR5,14535.22
303,1878318891,9/25/2023 13:33,Send Money,1071.84,#0127DWP5VM5,13463.38
304,1878318891,9/27/2023 10:18,Send Money,2166.31,#01D6P17HFQT,11297.07


# Calculating Coefficient of Variation of Balance

In [ ]:
# Define a function to calculate the coefficient of variation
def coefficient_of_variation(data):
    if len(data) < 2:
        return 0
    mean_balance = data[' Balance'].mean()
    std_balance = data[' Balance'].std()
    return (std_balance / mean_balance) * 100

# Calculate the coefficient of variation for each wallet's balances
cv_df = transaction_history_df.groupby('Wallet No').apply(coefficient_of_variation).reset_index(name='Balance Deviation (CV)')

mean_balance_dict = {}

# Iterate over the DataFrame and calculate the mean balance for each wallet number
for wallet_no, balances in transaction_history_df.groupby('Wallet No')[' Balance']:
    mean_balance = balances.mean()
    mean_balance_dict[wallet_no] = mean_balance

# Print the dictionary
print(mean_balance_dict)
cv_df

{1376318871: 6654.057204301074, 1376318942: 6155.060909090909, 1876318879: 3869.415217391305, 1878318891: 18244.073205128207, 1974318972: 3666.4169090909095}


,Wallet No,Balance Deviation (CV)
0,1376318871,79.131834
1,1376318942,31.957067
2,1876318879,66.538612
3,1878318891,23.577357
4,1974318972,81.363956


# Dividing each transaction types into multiple categories based on amount of transactions

In [ ]:
frequency_ranges = {
    'Send Money': {
        'Send Money_1': (10, 500),
        'Send Money_2': (501, 3000),
        'Send Money_3': (3001, 25000)
    },
    'Received Money': {
        'Received Money_1': (10, 500),
        'Received Money_2': (501, 3000),
        'Received Money_3': (3001, 25000)
    },
    'Cash In': {
        'Cash In_1': (10, 500),
        'Cash In_2': (501, 3000),
        'Cash In_3': (3001, 30000)
    },
    'Cash Out': {
        'Cash Out_1': (50, 500),
        'Cash Out_2': (501, 3000),
        'Cash Out_3': (3001, 25000)
    },
    'Make Payment': {
        'Make Payment_1': (1, 500),
        'Make Payment_2': (501, 3000),
        'Make Payment_3': (3001, 300000)
    },

    'Pay Bill': {
      'Pay Bill_1': (1, 2000),
      'Pay Bill_2': (2001, 300000)
    },

    'Add Money': {
      'Add Money_1': (50, 500),
      'Add Money_2': (501, 3000),
      'Add Money_3': (3001, 50000)
    },

    'Fund Transfer': {
      'Fund Transfer_1': (10, 500),
      'Fund Transfer_2': (501, 3000),
      'Fund Transfer_3': (3001, 50000)
    },

    'Request Money': {
      'Request Money': (10, 25000)
    },

    'Remittance': {
      'Remittance_1': (1, 10000),
      'Remittance_2': (10001, 125000)
    },

    'Donation': {
      'Donation_1': (10, 1500),
      'Donation_2': (1501, 300000)
    },

    'Others': {
      'Others_1': (1, 1500),
      'Others_2': (1501, 3000),
      'Others_3': (3001, 300000)
    },

}

# Calculating the frequencies of transactions under each category of transaction types for every customers

In [ ]:
# Iterate through each transaction type and count transactions within the specified ranges
for transaction_type, ranges in frequency_ranges.items():
    if transaction_type == 'Others':
        # Check for specific transaction types and count them within the 'Others' ranges
        for range_name, (min_amount, max_amount) in ranges.items():
            for wallet_no, group in transaction_history_df.groupby('Wallet No'):
                transaction_count = ((group['Transaction Type'].isin(['Land Tax', 'E-Porcha', 'DNCC Holding Tax', 'E-Mutation'])) & (group['Amount'] >= min_amount) & (group['Amount'] <= max_amount)).sum()
                cv_df.loc[cv_df['Wallet No'] == wallet_no, range_name] = transaction_count
    else:
        # For other transaction types, count transactions based on the transaction type
        for range_name, (min_amount, max_amount) in ranges.items():
            for wallet_no, group in transaction_history_df.groupby('Wallet No'):
                transaction_count = ((group['Transaction Type'] == transaction_type) & (group['Amount'] >= min_amount) & (group['Amount'] <= max_amount)).sum()
                cv_df.loc[cv_df['Wallet No'] == wallet_no, range_name] = transaction_count

# Convert the frequency columns to integer format
for column in cv_df.columns[2:]:
    cv_df[column] = cv_df[column].astype(int)

cv_df

,Wallet No,Balance Deviation (CV),Send Money_1,Send Money_2,Send Money_3,Received Money_1,Received Money_2,Received Money_3,Cash In_1,Cash In_2,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,1376318871,79.131834,3,10,2,2,14,3,1,10,...,0,0,4,3,0,0,0,3,1,0
1,1376318942,31.957067,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1876318879,66.538612,0,5,1,10,5,0,2,9,...,0,0,21,0,0,0,0,0,0,0
3,1878318891,23.577357,1,13,7,0,10,5,0,9,...,2,0,0,0,0,3,0,1,0,0
4,1974318972,81.363956,0,0,0,0,2,2,0,3,...,2,2,0,0,0,0,3,0,0,1


# Merge customer information with transactions. Shuffle the dataframe entries

In [ ]:
df = pd.merge(customer_info_df, cv_df, on='Wallet No', how='inner')
df.head()

,Name,Wallet No,Address,Gender,Balance,Age,Occupation,Loan Approval,Balance Deviation (CV),Send Money_1,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,Ali Bordhi Khan,1376318942,Rural,Male,5481.61,31,Engineer,UC,31.957067,0,...,0,0,0,0,0,0,0,0,1,0
1,Jagat Seth,1376318871,Urban,Male,8421.55,43,Business,Eligible,79.131834,3,...,0,0,4,3,0,0,0,3,1,0
2,Ghoseti Begum,1974318972,Urban,Female,212.59,45,NGO,Not Eligible,81.363956,0,...,2,2,0,0,0,0,3,0,0,1
3,Mir Jafar,1876318879,Urban,Male,6408.06,25,Student,UC,66.538612,0,...,0,0,21,0,0,0,0,0,0,0
4,Robert Clive,1878318891,Urban,Male,15840.49,54,Business,Eligible,23.577357,1,...,2,0,0,0,0,3,0,1,0,0


# Label Encoding

In [ ]:
loan_encode = LabelEncoder()

#Encode address
print(df["Address"].unique())
df['Address'] = loan_encode.fit_transform(df['Address'])
print(df["Address"].unique())

#Encode address
print(df["Occupation"].unique())
df['Occupation'] = loan_encode.fit_transform(df['Occupation'])
print(df["Occupation"].unique())

#Encode address
print(df["Loan Approval"].unique())
df['Loan Approval'] = loan_encode.fit_transform(df['Loan Approval'])
print(df["Loan Approval"].unique())

['Rural' 'Urban']
[0 1]
['Engineer' 'Business' 'NGO' 'Student']
[1 0 2 3]
['UC' 'Eligible' 'Not Eligible']
[2 0 1]


In [ ]:
df

,Name,Wallet No,Address,Gender,Balance,Age,Occupation,Loan Approval,Balance Deviation (CV),Send Money_1,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,Ali Bordhi Khan,1376318942,0,Male,5481.61,31,1,2,31.957067,0,...,0,0,0,0,0,0,0,0,1,0
1,Jagat Seth,1376318871,1,Male,8421.55,43,0,0,79.131834,3,...,0,0,4,3,0,0,0,3,1,0
2,Ghoseti Begum,1974318972,1,Female,212.59,45,2,1,81.363956,0,...,2,2,0,0,0,0,3,0,0,1
3,Mir Jafar,1876318879,1,Male,6408.06,25,3,2,66.538612,0,...,0,0,21,0,0,0,0,0,0,0
4,Robert Clive,1878318891,1,Male,15840.49,54,0,0,23.577357,1,...,2,0,0,0,0,3,0,1,0,0


# Dropping irrelevant columns

In [ ]:
columns_to_drop = ["Name", "Gender", "Loan Approval", "Wallet No"]
X = df.drop(columns = columns_to_drop, axis=1)
y = df["Loan Approval"]
X

,Address,Balance,Age,Occupation,Balance Deviation (CV),Send Money_1,Send Money_2,Send Money_3,Received Money_1,Received Money_2,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,0,5481.61,31,1,31.957067,0,0,0,0,2,...,0,0,0,0,0,0,0,0,1,0
1,1,8421.55,43,0,79.131834,3,10,2,2,14,...,0,0,4,3,0,0,0,3,1,0
2,1,212.59,45,2,81.363956,0,0,0,0,2,...,2,2,0,0,0,0,3,0,0,1
3,1,6408.06,25,3,66.538612,0,5,1,10,5,...,0,0,21,0,0,0,0,0,0,0
4,1,15840.49,54,0,23.577357,1,13,7,0,10,...,2,0,0,0,0,3,0,1,0,0


# Load Model

In [ ]:
with open('/content/drive/MyDrive/Upay/Upay_Loan_System_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Model Inference (Predictions)

In [ ]:
classes = {
    0:'Eligible',
    1:'Non Eligible',
    2:'Under Consideration'
}
for index, row in X.iterrows():
  print('============================-')
  row_entry = list(row)
  row_entry = [row_entry]
  # Get probability predictions for each class
  probability_predictions = model.predict_proba(row_entry)
  # # Convert the probability predictions to a percentage format
  percentage_predictions = [f"{prob * 100:.2f}%" for prob in probability_predictions[0]]

  # # Print the percentage predictions for each class
  class_labels = model.classes_
  for label, percentage in zip(class_labels, percentage_predictions):
      print(f"{classes[label]}: {percentage}")

Eligible: 14.25%
Non Eligible: 12.87%
Under Consideration: 72.88%
Eligible: 8.36%
Non Eligible: 0.04%
Under Consideration: 91.59%
Eligible: 0.01%
Non Eligible: 92.78%
Under Consideration: 7.21%
Eligible: 0.00%
Non Eligible: 0.00%
Under Consideration: 99.99%
Eligible: 99.10%
Non Eligible: 0.00%
Under Consideration: 0.90%


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
